<a href="https://colab.research.google.com/github/qkrwoghd04/binary_Classification_using_pytorch_with_ray_tune/blob/main/bert_classification_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
import sklearn
from tqdm.notebook import tqdm

In [46]:
train_df = pd.read_csv('./drive/MyDrive/image_dataset/processed/train_captions.csv')
test_df = pd.read_csv('./drive/MyDrive/image_dataset/processed/test_captions.csv')
train_df = train_df[['Caption','Label']]
test_df = test_df[['Caption', 'Label']]

In [47]:
train_df['Caption'] = train_df['Caption'].apply(lambda x: '' if pd.isna(x) else str(x))
test_df['Caption'] = test_df['Caption'].apply(lambda x: '' if pd.isna(x) else str(x))

In [48]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Label'], random_state=42)

In [49]:
train_df
all_labels = train_df['Label'].unique()
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

In [50]:
class CreateDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, label_to_idx):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        caption = row['Caption']
        label = row['Label']

        inputs = self.tokenizer.encode_plus(
            caption,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )

        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'targets': torch.tensor(self.label_to_idx[label], dtype=torch.long)
        }

In [51]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 256
BATCH_SIZE = 16

train_set = CreateDataset(train_df, tokenizer, MAX_LEN, label_to_idx)
val_set = CreateDataset(val_df, tokenizer, MAX_LEN, label_to_idx)
test_set = CreateDataset(test_df, tokenizer, MAX_LEN, label_to_idx)

In [52]:
class LabelClassifier(torch.nn.Module):
    def __init__(self):
        super(LabelClassifier, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.output = torch.nn.Linear(768, len(train_df['Label'].unique()))

    def forward(self, ids, mask):
        output = self.bert(ids, attention_mask=mask)
        output = self.dropout(output[0][:, 0, :])  # Use the [CLS] token representation
        output = self.output(output)
        return output

In [53]:
# 배치 크기 설정
BATCH_SIZE = 16

# train_loader 정의
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# val_loader 정의
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [64]:
model = LabelClassifier()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

LabelClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [65]:
EPOCHS = 20
LEARNING_RATE = 1e-5
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.CrossEntropyLoss()

In [66]:
for epoch in range(EPOCHS):
    model.train()
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} - Training")
    for batch in train_bar:
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        train_bar.set_postfix(loss=loss.item())

    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1} - Testing")
    for batch in val_bar:
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        targets = batch['targets'].to(device)

        with torch.no_grad():
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)

            val_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(val_loader)
    accuracy = total_correct / total_samples

    print(f"Epoch: {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1 - Training:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 - Testing:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.4106, Accuracy: 0.8250


Epoch 2 - Training:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 2 - Testing:   0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 19 - Testing:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 19, Loss: 0.6287, Accuracy: 0.8625


Epoch 20 - Training:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 20 - Testing:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 20, Loss: 0.6694, Accuracy: 0.8625


In [67]:
def test_accuracy(model, device="cpu"):
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            ids, mask, labels = data['ids'].to(device), data['mask'].to(device), data['targets'].to(device)
            outputs = model(ids, mask)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

In [68]:
test_accuracy(model, device)

0.9